# Champion Model

Champion Model: Support Vector Machine with specific features removal

Hyperparameters:
    'C': 1, 
    'gamma': 'scale',
    'kernel': 'rbf'


In [9]:
import numpy as np
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import f1_score

In [16]:
# Load data
X_train_input = joblib.load("Data/Xtrain1.pkl")
Y_train_input = np.load('Data/Ytrain1.npy')

# Load selected features indices from .npy file
selected_features = np.load("Validation/best_features_svm.npy")

# Extract patient IDs
patient_ids = X_train_input["Patient_Id"].values

# Split train/test by patient
mask_test = np.isin(patient_ids, [1, 2, 3])
mask_train = ~mask_test

X_train_df = X_train_input[mask_train]
Y_train = Y_train_input[mask_train]

X_test_df = X_train_input[mask_test]
Y_test = Y_train_input[mask_test]

# Extract selected features
X_train = np.vstack(X_train_df["Skeleton_Features"].values)[:, selected_features]
X_test = np.vstack(X_test_df["Skeleton_Features"].values)[:, selected_features]

# Champion hyperparameters
best_params = {
    "C": 1,
    "gamma": "scale",
    "kernel": "rbf",
}

# Build champion pipeline: scaler + SVM
champion_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(
        kernel=best_params["kernel"],
        C=best_params["C"],
        gamma=best_params["gamma"],
        degree=best_params.get("degree", 3),
        probability=False,
        random_state=42
    ))
])

# Fit pipeline
champion_pipeline.fit(X_train, Y_train)

,steps,"[('scaler', ...), ('svm', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,C,1
,kernel,'rbf'
,degree,3
,gamma,'scale'


In [17]:
# Predict on test patients
Y_test_pred = champion_pipeline.predict(X_test)

# Compute F1-score (macro for multi-class)
test_f1 = f1_score(Y_test, Y_test_pred, average='macro')
print("Champion Model F1-score on test set (patients 1,2,3):", test_f1)

Champion Model F1-score on test set (patients 1,2,3): 0.8976982097186701


# Train and Save Champion Model


In [18]:
# Load Data
X_train_input = joblib.load("Data/Xtrain1.pkl")
Y_train_input = np.load('Data/Ytrain1.npy')

# Load selected features indices from .npy file
selected_features = np.load("Validation/best_features_svm.npy")

# Extract features
X_train = np.vstack(X_train_input["Skeleton_Features"].values)[:, selected_features]
Y_train = Y_train_input

In [19]:
# Champion hyperparameters
best_params = {
    "C": 1,
    "gamma": "scale",
    "kernel": "rbf",
}

# Build champion pipeline:
champion_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(
        kernel=best_params["kernel"],
        C=best_params["C"],
        gamma=best_params["gamma"],
        degree=best_params.get("degree", 3),
        probability=False,
        random_state=42
    ))
])

In [20]:
# Fit pipeline
champion_pipeline.fit(X_train, Y_train)

# Save pipeline
joblib.dump(champion_pipeline, "champion.pkl")
print("Champion SVM pipeline saved as champion.pkl")

Champion SVM pipeline saved as champion.pkl
